In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf
import Chroma_db_functions as Cf

from apikey import apikey 

from langchain.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

from chromadb.config import Settings
import uuid

In [2]:
persist_directory = 'chroma-db'
embedding = Cf.get_embeddings()

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\juand\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\

In [3]:
db = Chroma(persist_directory= persist_directory, embedding_function=embedding,collection_name='encodings', )

In [4]:
#Ensure we are pulling data correctly
db.similarity_search(query='hello')

[Document(page_content='Brand: HM / Type: Dress   / Fit: Regular fit   / Sleeve style: Long sleeve   / Neckline: V-neck   / Material: Cotton   / Formality: Casual   / Seasson: Autumn   / Colors: 100% Black   / Texture: Smooth   / Transparency: Opaque   / Details and Embellishments: Buttons   / Shape: Fitted   / Length: Midi   / Collar Style: Not applicable   / Sleeve Style: Long   / Patterns: Not applicable   / Patterns placement: Not applicable   / Fluidity of fabric: Moderate   / Fabric weight: Medium   / Pocket Presence: No   / Pocket placement: Not applicable   / Pocket size: Not applicable   / Breathability: Medium   / Occasion Suitability: Casual   / Lapel: Not applicable', metadata={'source': '19'}),
 Document(page_content='Brand: HM / Type: Dress / Fit: Regular fit / Sleeve style: Long sleeve / Neckline: V-neck / Material: Polyester / Formality: Casual / Season: Autumn, Spring / Colors: Black 90%, White 10% / Texture: Smooth / Transparency: Opaque / Details and Embellishments: 

In [5]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [6]:
os.environ['OPENAI_API_KEY'] = apikey

In [19]:
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Test chain 
Setting up -> https://python.langchain.com/v0.2/docs/tutorials/rag/

In [23]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [32]:
from langchain import hub

prompt_hub = hub.pull("rlm/rag-prompt")

In [33]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [34]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_hub
    | turbo_llm
    | StrOutputParser()
)

In [29]:
### lets set up the system

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [48]:
system_prompt = (
    "You are a search engine for clothing. "
    "Use the following pieces of retrieved context to suggest the best peace of clothing "
    "the question. If you don't know the answer, say that you "
    "answer concise."
    "Explain why the 5 selected options of the context are the best ones"
    "\n\n"
    "{context}"
)

In [49]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [50]:
question_answer_chain = create_stuff_documents_chain(turbo_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [42]:
query = "Brand: HM / Type: Dress   / Fit: Regular fit   / Sleeve style: Long sleeve   / Neckline: V-neck   / Material: Cotton   / Formality: Casual   / Seasson: Autumn   / Colors: 100% Black   / Texture: Smooth   / Transparency: Opaque   / Details and Embellishments: Buttons   / Shape: Fitted   / Length: Midi   / Collar Style: Not applicable   / Sleeve Style: Long   / Patterns: Not applicable   / Patterns placement: Not applicable   / Fluidity of fabric: Moderate   / Fabric weight: Medium   / Pocket Presence: No   / Pocket placement: Not applicable   / Pocket size: Not applicable   / Breathability: Medium   / Occasion Suitability: Casual   / Lapel: Not applicable'"

In [51]:
query = "/ Material: Cotton   / Formality: Casual   / Seasson: Autumn   / Colors: 100% Black   / Texture: Smooth   / Transparency: Opaque   / Details and Embellishments: Buttons / Sleeve Style: Long / Patterns placement: Not applicable   / Fluidity of fabric: Moderate   / Fabric weight: Medium   / Pocket Presence: No   / Pocket placement: Not applicable   / Pocket size: Not applicable   / Breathability: Medium   / Occasion Suitability: Casual   / Lapel: Not applicable'"

In [52]:
response = rag_chain.invoke({"input": query})
print(response["answer"])

Based on the provided context, the best option for you would be the dress described as follows:

- Brand: HM
- Type: Dress
- Fit: Regular fit
- Sleeve style: Long sleeve
- Neckline: V-neck
- Material: Cotton
- Formality: Casual
- Season: Autumn
- Color: Black
- Texture: Smooth
- Transparency: Opaque
- Details and Embellishments: Buttons
- Shape: Fitted
- Length: Midi
- Sleeve Style: Long
- Patterns: Not applicable
- Fluidity of fabric: Moderate
- Fabric weight: Medium
- Pocket Presence: No
- Pocket placement: Not applicable
- Pocket size: Not applicable
- Breathability: Medium
- Occasion Suitability: Casual
- Lapel: Not applicable

This dress is a great choice for autumn due to its cotton material, which provides moderate breathability and medium fabric weight, making it suitable for the season. The black color, smooth texture, and fitted shape add to its casual formality, while the V-neck and long sleeves make it versatile for various occasions. The absence of patterns and pockets kee

In [53]:
for document in response["context"]:
    print(document)
    print()

page_content='Brand: HM / Type: Dress / Fit: Regular fit / Sleeve style: Long sleeve / Neckline: V-neck / Material: Likely polyester or a blend / Formality: Casual, Business Casual / Season: Autumn, Spring / Colors: Black (80%), White (20%) / Texture: Smooth / Transparency: Opaque / Details and Embellishments: Buttons / Shape: Fitted / Length: Midi / Collar Style: Not applicable / Sleeve Style: Long / Patterns: Polka dots / Patterns placement: Throughout / Fluidity of fabric: Medium / Fabric weight: Medium / Pocket Presence: No / Pocket placement: Not applicable / Pocket size: Not applicable / Breathability: Medium / Occasion Suitability: Casual, business casual / Lapel: Not applicable' metadata={'source': '17'}

page_content='Brand: HM / Type: Dress / Fit: Regular fit, Loose / Sleeve style: Long sleeve / Neckline: V-neck / Material: Polyester / Formality: Casual, Business Casual / Season: Autumn, Spring / Colors: Black (90%), White (10%) / Texture: Smooth / Transparency: Opaque / Deta

# testing 2nd

In [43]:
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [44]:
persist_directory = 'chroma-db-full-description'
embedding = Cf.get_embeddings()

In [45]:
vectorstore = Chroma(
    collection_name="multi_modal_rag", embedding_function=embedding,persist_directory = persist_directory
)

In [46]:
vectorstore.similarity_search('summer')

[Document(page_content="The shirt in the images is a Gucci cotton gabardine piece that exudes an effortlessly chic summer vibe, inspired by the aesthetic of beach clubs. It features a relaxed fit with short sleeves, ideal for warm weather, and a classic camp collar that adds to its laid-back yet sophisticated style. The material is high-quality cotton gabardine, known for its durability and smooth texture, making it both breathable and comfortable for summer wear. The fabric is of medium weight, ensuring it drapes nicely without being too heavy or too light. The primary color is a light beige, complemented by a subtle, repeated GG pattern in a slightly darker shade of beige, adding a touch of luxury and brand recognition without overwhelming the design. The pattern is uniformly placed across the entire shirt, maintaining a consistent look. The shirt includes two front chest pockets with flaps, providing both style and functionality, while enhancing its casual appeal. There are no lapel

In [47]:
store = InMemoryStore()
id_key = 'doc_id'

In [48]:
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

In [49]:
#retriever = MultiVectorRetriever(
#    vectorstore=vectorstore,
#    docstore=store,
#    id_key=id_key,
#    search_kwargs={"k":5}
#)

In [50]:
docs = retriever.invoke(
    "Gucci"
)

In [74]:
#docs

In [52]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [53]:
from langchain import hub

prompt_hub = hub.pull("rlm/rag-prompt")

In [54]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [55]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_hub
    | turbo_llm
    | StrOutputParser()
)

In [56]:
### lets set up the system

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [69]:
system_prompt = (
    "You are a search engine for clothing. "
    "Use the following pieces of retrieved context to suggest the best peace of clothing "
    "the question. If you don't know the answer, say that you "
    "Explain each of the 5 selected options of the context are the best ones in a consice matter"
    "\n\n"
    "{context}"
)

In [70]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [71]:
question_answer_chain = create_stuff_documents_chain(turbo_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [72]:
query = """

find piece that could be similar to this description:

This silk shirt features an optical monogram print with a regular fit and long sleeves ending in buttoned cuffs. It has a classic pointed collar that adds a touch of formality. The high-quality silk fabric provides a luxurious feel and fluid drape, making it suitable for all seasons due to its breathable and lightweight nature. The deep lavender and white colors are intricately woven to create a striking visual effect, with consistent pattern placement for a cohesive look. The smooth, slightly glossy texture typical of silk, together with its non-transparent quality, makes the shirt appropriate for both formal and semi-formal occasions. Absent of pockets, embellishments, or lapels, the design maintains a sleek and streamlined silhouette. The shirt has a standard length that typically falls just below the waistline and is finished with concealed buttons for a minimalist aesthetic.

"""

In [73]:
response = rag_chain.invoke({"input": query})
print(response["answer"])

The best piece of clothing that could be similar to the description provided would be a refined silk shirt with an optical monogram print, regular fit, long sleeves, pointed collar, high-quality silk fabric, deep lavender and white colors, consistent pattern placement, smooth texture, non-transparent quality, suitable for formal and semi-formal occasions, absent of pockets, embellishments, or lapels, standard length falling just below the waistline, and finished with concealed buttons for a minimalist aesthetic.
